In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def load_and_parse_timestamps(file_path, timestamp_col):
    """Load CSV and parse timestamps with mixed format support."""
    print(f"📂 Loading {file_path}...")
    df = pd.read_csv(file_path)
    
    # Show samples
    print(f"   Sample timestamps:")
    print(f"   {df[timestamp_col].head(3).values}")
    
    # Parse with multiple strategies
    try:
        df[timestamp_col] = pd.to_datetime(df[timestamp_col], 
                                          format='ISO8601', 
                                          errors="coerce")
        
        if df[timestamp_col].isna().sum() > len(df) * 0.5:
            df[timestamp_col] = pd.to_datetime(df[timestamp_col], 
                                              format='mixed', 
                                              dayfirst=True, 
                                              errors="coerce")
    except:
        df[timestamp_col] = pd.to_datetime(df[timestamp_col], 
                                          dayfirst=True, 
                                          errors="coerce")
    
    # Check parsing success
    failed = df[timestamp_col].isna().sum()
    if failed > 0:
        print(f"   ⚠️  {failed} timestamps failed to parse")
    else:
        print(f"   ✅ All timestamps parsed")
    
    # Sort by timestamp
    df = df.sort_values(timestamp_col).reset_index(drop=True)
    
    print(f"✅ Loaded {len(df)} rows")
    print(f"   Range: {df[timestamp_col].min()} to {df[timestamp_col].max()}")
    
    return df

def find_missing_timestamps(df, timestamp_col, frequency):
    """Find missing timestamps in the dataset."""
    print(f"\n🔍 Checking for missing timestamps...")
    
    timestamps = df[timestamp_col].dropna()
    
    # Generate expected full range
    expected = pd.date_range(
        start=timestamps.min(),
        end=timestamps.max(),
        freq=frequency
    )
    
    # Find missing
    actual_set = set(timestamps)
    missing_set = set(expected) - actual_set
    missing_list = sorted(list(missing_set))
    
    print(f"\n📊 TIMESTAMP ANALYSIS")
    print("=" * 70)
    print(f"Expected timestamps: {len(expected)}")
    print(f"Actual timestamps: {len(timestamps)}")
    print(f"Missing timestamps: {len(missing_list)}")
    print(f"Completeness: {(len(timestamps) / len(expected) * 100):.2f}%")
    
    return missing_list, expected

def analyze_gaps(missing_list, frequency):
    """Analyze gap patterns in missing timestamps."""
    if len(missing_list) == 0:
        print(f"\n✅ NO MISSING TIMESTAMPS FOUND!")
        return None
    
    print(f"\n🚨 MISSING TIMESTAMP DETAILS")
    print("=" * 70)
    
    # Find consecutive gaps
    gaps = []
    if len(missing_list) > 0:
        current_gap_start = missing_list[0]
        current_gap_size = 1
        
        for i in range(1, len(missing_list)):
            if (missing_list[i] - missing_list[i-1]).total_seconds() == pd.Timedelta(frequency).total_seconds():
                current_gap_size += 1
            else:
                gaps.append({
                    'start': current_gap_start,
                    'end': missing_list[i-1],
                    'size': current_gap_size
                })
                current_gap_start = missing_list[i]
                current_gap_size = 1
        
        gaps.append({
            'start': current_gap_start,
            'end': missing_list[-1],
            'size': current_gap_size
        })
    
    # Show largest gaps
    gaps_sorted = sorted(gaps, key=lambda x: x['size'], reverse=True)
    
    print(f"\n📍 Top 10 Largest Gaps:")
    for i, gap in enumerate(gaps_sorted[:10], 1):
        duration = gap['size'] * pd.Timedelta(frequency)
        print(f"   {i}. {gap['start']} to {gap['end']}")
        print(f"      Size: {gap['size']} intervals ({duration})")
    
    return gaps_sorted

def save_missing_report(missing_list, output_file=None, input_file="input.csv"):
    """Save missing timestamps to CSV."""
    if len(missing_list) == 0:
        print(f"\n✅ Nothing to save - no missing timestamps")
        return None
    
    if output_file is None:
        stem = Path(input_file).stem
        output_file = f"{stem}_missing_timestamps.csv"
    
    df_missing = pd.DataFrame({
        'Missing_Timestamp': missing_list
    })
    
    df_missing.to_csv(output_file, index=False)
    print(f"\n💾 Missing timestamps saved to: {output_file}")
    
    return output_file

def fill_missing_timestamps(df, timestamp_col, frequency, method="week_back"):
    """Fill missing timestamps with data from previous periods.
    
    Methods:
    - 'day_back': Use data from 24 hours ago
    - 'week_back': Use data from 7 days ago (default)
    - 'month_back': Use data from 30 days ago
    - 'year_back': Use data from 365 days ago
    - 'forward_fill': Use last available value
    - 'interpolate': Linear interpolation between values
    """
    print(f"\n📝 Filling missing timestamps with '{method}' method...")
    
    df_filled = df.copy()
    df_filled = df_filled.set_index(timestamp_col).asfreq(frequency).reset_index()
    
    # Calculate shift periods based on frequency
    td = pd.Timedelta(frequency).total_seconds() / 60  # minutes per interval
    
    shift_periods = {
        'day_back': int((24 * 60) / td),
        'week_back': int((7 * 24 * 60) / td),
        'month_back': int((30 * 24 * 60) / td),
        'year_back': int((365 * 24 * 60) / td),
    }
    
    numeric_cols = df_filled.select_dtypes(include=[np.number]).columns
    
    if method in shift_periods:
        shift = shift_periods[method]
        print(f"   Shifting by {shift} periods ({method})")
        
        for col in numeric_cols:
            missing_mask = df_filled[col].isna()
            shifted_col = df_filled[col].shift(shift)
            df_filled.loc[missing_mask, col] = shifted_col[missing_mask]
        
        # Forward fill any remaining NaNs from the shift method
        for col in numeric_cols:
            df_filled[col] = df_filled[col].ffill()
    
    elif method == 'forward_fill':
        print(f"   Using forward fill")
        for col in numeric_cols:
            df_filled[col] = df_filled[col].ffill()
    
    elif method == 'interpolate':
        print(f"   Using linear interpolation")
        for col in numeric_cols:
            df_filled[col] = df_filled[col].interpolate(method='linear')
    
    # Count filled values
    original_nulls = df[numeric_cols].isna().sum().sum()
    filled_nulls = df_filled[numeric_cols].isna().sum().sum()
    filled_count = original_nulls - filled_nulls
    
    if filled_count == 0 and original_nulls == 0:
        print(f"   ✅ No missing values found - data is complete!")
    else:
        print(f"   ✅ Filled {filled_count} missing values")
        print(f"   Remaining NaN: {filled_nulls}")
    
    return df_filled

# Missing Timestamp Checker

Identify and analyze missing timestamps in your time-series data.

## Quick Start

1. **Cell 2** - Set configuration (input file, frequency)
2. **Cell 3** - Run the checker
3. **Cell 4** - View detailed analysis

## Features

✅ Detect missing timestamps  
✅ Show gap locations and sizes  
✅ Generate report with statistics  
✅ Handle mixed timestamp formats  
✅ Export missing timestamps to CSV  

## Configuration

| Setting | Description |
|---------|------------|
| INPUT_FILE | CSV file to check |
| FREQUENCY | "15min", "1H", "1D" (15-minute, hourly, daily) |
| TIMESTAMP_COL | Column name with timestamps |

In [5]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# 📋 USER CONFIGURATION - Modify these settings
INPUT_FILE = "2026_proper_data.csv"      # CSV file to analyze
TIMESTAMP_COL = "Timestamp"                    # Column name with timestamps
FREQUENCY = "15min"                            # "15min" for 15-minute intervals, "1H" for hourly, "1D" for daily
OUTPUT_MISSING = True                          # Save missing timestamps to CSV file
FILL_METHOD = "month_back"                      # "day_back", "week_back", "month_back", "year_back", "forward_fill", "interpolate"
SAVE_FILLED = True                             # Save filled dataset to CSV

print(f"⚙️  CONFIGURATION")
print(f"   Input file: {INPUT_FILE}")
print(f"   Timestamp column: {TIMESTAMP_COL}")
print(f"   Frequency: {FREQUENCY}")
print(f"   Fill method: {FILL_METHOD}")
print(f"   Export missing: {OUTPUT_MISSING}")
print(f"   Save filled data: {SAVE_FILLED}")


# ============================================================================
# MAIN PIPELINE
# ============================================================================

print("\n" + "="*70)
print("MISSING TIMESTAMP CHECKER & FILLER")
print("="*70)

# Step 1: Load data
df = load_and_parse_timestamps(INPUT_FILE, TIMESTAMP_COL)

# Step 2: Find missing
missing_timestamps, expected_timestamps = find_missing_timestamps(df, TIMESTAMP_COL, FREQUENCY)

# Step 3: Analyze gaps
gaps = analyze_gaps(missing_timestamps, FREQUENCY)

# Step 4: Save missing report
if OUTPUT_MISSING and len(missing_timestamps) > 0:
    save_missing_report(missing_timestamps, input_file=INPUT_FILE)

# Step 5: Fill missing timestamps
df_filled = fill_missing_timestamps(df, TIMESTAMP_COL, FREQUENCY, method=FILL_METHOD)

# Step 6: Save filled dataset
if SAVE_FILLED:
    stem = Path(INPUT_FILE).stem
    output_filled = f"{stem}_filled.csv"
    df_filled.to_csv(output_filled, index=False)
    print(f"\n💾 Filled dataset saved to: {output_filled}")

print(f"\n✨ Analysis and filling complete!")


⚙️  CONFIGURATION
   Input file: 2026_proper_data.csv
   Timestamp column: Timestamp
   Frequency: 15min
   Fill method: month_back
   Export missing: True
   Save filled data: True

MISSING TIMESTAMP CHECKER & FILLER
📂 Loading 2026_proper_data.csv...
   Sample timestamps:
   ['2017-01-01 00:00:00' '2017-01-01 00:15:00' '2017-01-01 00:30:00']
   ✅ All timestamps parsed
✅ Loaded 316933 rows
   Range: 2017-01-01 00:00:00 to 2026-01-15 09:00:00

🔍 Checking for missing timestamps...

📊 TIMESTAMP ANALYSIS
Expected timestamps: 316933
Actual timestamps: 316933
Missing timestamps: 0
Completeness: 100.00%

✅ NO MISSING TIMESTAMPS FOUND!

📝 Filling missing timestamps with 'month_back' method...
   Shifting by 2880 periods (month_back)
   ✅ No missing values found - data is complete!

💾 Filled dataset saved to: 2026_proper_data_filled.csv

✨ Analysis and filling complete!
